In [1]:
pip install modin[ray] -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.0 MB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
#modin pandas is faster than pandas
import modin.pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd

In [4]:
#Dask is used for reading the csv. When I was using pandas in the other projects sometimes pandas cannot read all of the data. For that reason I prefer to use dask.
sales_df = dd.read_csv("transactions.csv")
sales_df = sales_df.compute()
sales_df.drop(index=0,inplace=True) #There is only one sales info at 2013-01-01. I am deleting this data because of that.
sales_df["date"] = pd.to_datetime(sales_df["date"]) #Converting date column to datetime format
#Gathering all transactions of the all stores for each day.
sales_df = sales_df.groupby("date",as_index=False).agg({"transactions":"sum"})
#I added some columns for deep learning and machine learning techniques.
#Firstly, national holidays are added.
holidays_df = dd.read_csv("holidays_events.csv")
holidays_df = holidays_df.compute()
holidays_df.drop(columns=["type","locale_name","description","transferred"],inplace=True)
holidays_df = holidays_df[holidays_df['locale'] == 'National']
sales_df['is_holiday'] = sales_df['date'].isin(holidays_df['date']).astype(int)
#Here, I added a column to the table that contains the average up to a specific day in the past
for window in [3, 7, 30]:
    sales_df[f'past_rolling_mean_{window}'] = sales_df['transactions'].rolling(window).mean().shift(1)

#Creating lags
for lag in range(1, 61):
    sales_df[f'lag_{lag}'] = sales_df['transactions'].shift(lag)

#Seperating date to year, month and day
sales_df['year'] = sales_df['date'].dt.year
sales_df['month'] = sales_df['date'].dt.month
sales_df['day'] = sales_df['date'].dt.day

In [5]:
sales_df.dropna(inplace=True)
sales_df

,date,transactions,is_holiday,past_rolling_mean_3,past_rolling_mean_7,past_rolling_mean_30,lag_1,lag_2,lag_3,lag_4,...,lag_54,lag_55,lag_56,lag_57,lag_58,lag_59,lag_60,year,month,day
60,2013-03-03,92716,0,82652.666667,77589.428571,78258.500000,98616.0,79524.0,69818.0,70170.0,...,72325.0,75597.0,90464.0,93573.0,78494.0,78504.0,93215.0,2013,3,3
61,2013-03-04,79078,0,90285.333333,78816.714286,78738.966667,92716.0,98616.0,79524.0,69818.0,...,71971.0,72325.0,75597.0,90464.0,93573.0,78494.0,78504.0,2013,3,4
62,2013-03-05,75134,0,90136.666667,79804.000000,78130.000000,79078.0,92716.0,98616.0,79524.0,...,66383.0,71971.0,72325.0,75597.0,90464.0,93573.0,78494.0,2013,3,5
63,2013-03-06,76519,0,82309.333333,80722.285714,77733.600000,75134.0,79078.0,92716.0,98616.0,...,70338.0,66383.0,71971.0,72325.0,75597.0,90464.0,93573.0,2013,3,6
64,2013-03-07,73910,0,76910.333333,81629.285714,77773.233333,76519.0,75134.0,79078.0,92716.0,...,85511.0,70338.0,66383.0,71971.0,72325.0,75597.0,90464.0,2013,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676,2017-08-11,89551,1,83979.333333,89655.000000,88652.166667,83064.0,85452.0,83422.0,87072.0,...,99193.0,114201.0,92852.0,81831.0,82752.0,81063.0,81080.0,2017,8,11
1677,2017-08-12,89927,0,86022.333333,89404.428571,88832.033333,89551.0,83064.0,85452.0,83422.0,...,82891.0,99193.0,114201.0,92852.0,81831.0,82752.0,81063.0,2017,8,12
1678,2017-08-13,85993,0,87514.000000,87807.142857,89205.600000,89927.0,89551.0,83064.0,85452.0,...,83508.0,82891.0,99193.0,114201.0,92852.0,81831.0,82752.0,2017,8,13
1679,2017-08-14,85448,0,88490.333333,86354.428571,89149.666667,85993.0,89927.0,89551.0,83064.0,...,82929.0,83508.0,82891.0,99193.0,114201.0,92852.0,81831.0,2017,8,14


# Method 2 One-step Ahead Forecasting
Bu methodta method 1'den farklı olarak önümüzdeki çoklu gün tahmininden ziyade sadece önümüzdeki tek günü tahmin etmeye çalışıyoruz.

Örneğin, veri seti 1500'e kadar train ediliyor ve tahmin olarka sadece 1 gün 1501. gün oluyor.

Bu methodta validation kullanılmayacaktır.

Burada 2 alt teknik kullanacağız.
- Modelin Tekrar Eğitilmesi
- Recursive Forecasting


In [59]:
#This function is created to split dataset as train, validation and test.
def split_dataset(df, predict_day, seq_length=30):
    predict_day -= 1
    train_df = df.iloc[:predict_day]
    test_df = df.iloc[predict_day - seq_length:predict_day + 1]
    return train_df, test_df

In [60]:
train_df, test_df = split_dataset(sales_df,1551)

In [30]:
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [62]:
train_df

,date,transactions,is_holiday,past_rolling_mean_3,past_rolling_mean_7,past_rolling_mean_30,lag_1,lag_2,lag_3,lag_4,...,lag_54,lag_55,lag_56,lag_57,lag_58,lag_59,lag_60,year,month,day
60,2013-03-03,92716,0,82652.666667,77589.428571,78258.500000,98616.0,79524.0,69818.0,70170.0,...,72325.0,75597.0,90464.0,93573.0,78494.0,78504.0,93215.0,2013,3,3
61,2013-03-04,79078,0,90285.333333,78816.714286,78738.966667,92716.0,98616.0,79524.0,69818.0,...,71971.0,72325.0,75597.0,90464.0,93573.0,78494.0,78504.0,2013,3,4
62,2013-03-05,75134,0,90136.666667,79804.000000,78130.000000,79078.0,92716.0,98616.0,79524.0,...,66383.0,71971.0,72325.0,75597.0,90464.0,93573.0,78494.0,2013,3,5
63,2013-03-06,76519,0,82309.333333,80722.285714,77733.600000,75134.0,79078.0,92716.0,98616.0,...,70338.0,66383.0,71971.0,72325.0,75597.0,90464.0,93573.0,2013,3,6
64,2013-03-07,73910,0,76910.333333,81629.285714,77773.233333,76519.0,75134.0,79078.0,92716.0,...,85511.0,70338.0,66383.0,71971.0,72325.0,75597.0,90464.0,2013,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,2017-06-01,90171,0,86565.333333,89447.285714,89848.133333,93365.0,83210.0,83121.0,94970.0,...,98033.0,85781.0,80775.0,85255.0,84872.0,81294.0,94696.0,2017,6,1
1606,2017-06-02,91840,0,88915.333333,91114.428571,89898.233333,90171.0,93365.0,83210.0,83121.0,...,94617.0,98033.0,85781.0,80775.0,85255.0,84872.0,81294.0,2017,6,2
1607,2017-06-03,106965,0,91792.000000,90455.142857,89969.133333,91840.0,90171.0,93365.0,83210.0,...,81868.0,94617.0,98033.0,85781.0,80775.0,85255.0,84872.0,2017,6,3
1608,2017-06-04,106972,0,96325.333333,91948.857143,90802.733333,106965.0,91840.0,90171.0,93365.0,...,84105.0,81868.0,94617.0,98033.0,85781.0,80775.0,85255.0,2017,6,4


In [61]:
test_df

,date,transactions,is_holiday,past_rolling_mean_3,past_rolling_mean_7,past_rolling_mean_30,lag_1,lag_2,lag_3,lag_4,...,lag_54,lag_55,lag_56,lag_57,lag_58,lag_59,lag_60,year,month,day
1580,2017-05-07,103210,0,90497.666667,93261.142857,89471.566667,104203.0,85333.0,81957.0,89713.0,...,78200.0,79887.0,95842.0,97521.0,84691.0,76926.0,86740.0,2017,5,7
1581,2017-05-08,84221,0,97582.000000,94217.714286,90052.533333,103210.0,104203.0,85333.0,81957.0,...,84574.0,78200.0,79887.0,95842.0,97521.0,84691.0,76926.0,2017,5,8
1582,2017-05-09,82345,0,97211.333333,91043.571429,89592.133333,84221.0,103210.0,104203.0,85333.0,...,79030.0,84574.0,78200.0,79887.0,95842.0,97521.0,84691.0,2017,5,9
1583,2017-05-10,83637,0,89925.333333,90140.285714,89183.066667,82345.0,84221.0,103210.0,104203.0,...,87303.0,79030.0,84574.0,78200.0,79887.0,95842.0,97521.0,2017,5,10
1584,2017-05-11,81476,0,83401.000000,89272.285714,89242.033333,83637.0,82345.0,84221.0,103210.0,...,99094.0,87303.0,79030.0,84574.0,78200.0,79887.0,95842.0,2017,5,11
1585,2017-05-12,91482,0,82486.000000,89203.571429,89154.400000,81476.0,83637.0,82345.0,84221.0,...,96738.0,99094.0,87303.0,79030.0,84574.0,78200.0,79887.0,2017,5,12
1586,2017-05-13,121889,1,85531.666667,90082.000000,89402.066667,91482.0,81476.0,83637.0,82345.0,...,85005.0,96738.0,99094.0,87303.0,79030.0,84574.0,78200.0,2017,5,13
1587,2017-05-14,101257,1,98282.333333,92608.571429,90439.966667,121889.0,91482.0,81476.0,83637.0,...,81716.0,85005.0,96738.0,99094.0,87303.0,79030.0,84574.0,2017,5,14
1588,2017-05-15,83716,0,104876.000000,92329.571429,90931.300000,101257.0,121889.0,91482.0,81476.0,...,80140.0,81716.0,85005.0,96738.0,99094.0,87303.0,79030.0,2017,5,15
1589,2017-05-16,85528,0,102287.333333,92257.428571,90483.000000,83716.0,101257.0,121889.0,91482.0,...,77989.0,80140.0,81716.0,85005.0,96738.0,99094.0,87303.0,2017,5,16


In [67]:
seq_length = 30
X_train, y_train = create_sequences(train_df['transactions'].values, seq_length)
X_test, y_test = create_sequences(test_df['transactions'].values, seq_length)

In [68]:
X_train

array([[ 92716,  79078,  75134, ...,  90611,  82563,  81678],
       [ 79078,  75134,  76519, ...,  82563,  81678,  77165],
       [ 75134,  76519,  73910, ...,  81678,  77165,  78534],
       ...,
       [ 81957,  85333, 104203, ...,  93365,  90171,  91840],
       [ 85333, 104203, 103210, ...,  90171,  91840, 106965],
       [104203, 103210,  84221, ...,  91840, 106965, 106972]])

In [69]:
y_train

array([ 77165,  78534,  72217, ..., 106965, 106972,  88273])

In [70]:
y_test

array([84447])

In [71]:
from sklearn.preprocessing import StandardScaler
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)
X_test_scaled = scaler_X.transform(X_test)

In [72]:
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
y_val_scaled = scaler_y.fit_transform(y_val.reshape(-1, 1))

In [73]:
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_val_scaled = X_val_scaled.reshape((X_val_scaled.shape[0], X_val_scaled.shape[1], 1))
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

## Recursive Forecasting




### LSTM Model

In [93]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.models import Model

In [75]:
from tensorflow.keras.layers import SimpleRNN

In [95]:
def model_training(model: Model, start_day: int, end_day: int, model_name: str):
  y_tests = []
  y_predictions = []
  for day in range(start_day,end_day):
      train_df, test_df = split_dataset(sales_df,day)
      X_train, y_train = create_sequences(train_df['transactions'].values, seq_length)
      X_test, y_test = create_sequences(test_df['transactions'].values, seq_length)

      scaler_X = StandardScaler()
      X_train_scaled = scaler_X.fit_transform(X_train)
      X_test_scaled = scaler_X.transform(X_test)

      X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
      X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

      scaler_y = StandardScaler()
      y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))

      model.fit(
            X_train_scaled, y_train_scaled,
            epochs=100,
            batch_size=32,
            verbose=0
        )

      y_pred_scaled = model.predict(X_test_scaled)
      y_pred_scaled = scaler_y.inverse_transform(y_pred_scaled)
      y_test = y_test.reshape(-1,1)

      y_tests.append(y_test.item())
      y_predictions.append(y_pred_scaled.item())
  return y_tests, y_predictions

In [84]:
model_LSTM_simple = Sequential([
    LSTM(32, input_shape=(X_train_scaled.shape[1], 1), kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(1)
])
model_LSTM_simple.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])

history_simple = model_LSTM_simple.fit(
    X_train_scaled, y_train_scaled,
    epochs=100,
    batch_size=32,
    verbose=0
)

Epoch 1/100


48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.9113 - mae: 0.6820
Epoch 2/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.7431 - mae: 0.6402
Epoch 3/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7689 - mae: 0.6413
Epoch 4/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.8052 - mae: 0.6375
Epoch 5/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6720 - mae: 0.6119
Epoch 6/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6979 - mae: 0.6227
Epoch 7/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6920 - mae: 0.5796
Epoch 8/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6123 - mae: 0.5409
Epoch 9/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4803 - mae: 0.4865
Epoch 10/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5258 - mae: 0.4715
Epoch 11/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4522 - mae: 0.4325
Epoch 12/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4760 - mae: 0.4370
Epoch 13/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/st

In [96]:
deneme_tests, deneme_preds = model_training(model_LSTM_simple,1500,1512,"Halo")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


In [97]:
deneme_tests

[90664,
 83043,
 80334,
 78999,
 82905,
 90055,
 101125,
 101186,
 81901,
 82193,
 84709,
 78971]

In [98]:
deneme_preds

[95181.5078125,
 83618.5390625,
 83201.6171875,
 80380.0234375,
 76658.609375,
 84495.71875,
 99346.5546875,
 96128.546875,
 88449.6796875,
 81866.2109375,
 83502.328125,
 81413.7734375]

In [99]:
rmse_lstm_simple = np.sqrt(mean_squared_error(deneme_tests, deneme_preds))
mape_lstm_simple = mean_absolute_percentage_error(deneme_tests, deneme_preds)
r2_lstm_simple = r2_score(deneme_tests, deneme_preds)
print(f"R^2 is {r2_lstm_simple} \n RMSE is {rmse_lstm_simple} \n MAPE is {mape_lstm_simple}")

R^2 is 0.7345871327181455 
 RMSE is 3872.428992645345 
 MAPE is 0.03697428535774474


In [88]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
y_pred_scaled = model_LSTM_simple.predict(X_test_scaled)
y_pred_scaled = scaler_y.inverse_transform(y_pred_scaled)
y_test = y_test.reshape(-1,1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
R^2 is nan 
 RMSE is 3826.3281354321925 
 MAPE is 0.045310407876968384


In [ ]:
rmse_lstm_simple = np.sqrt(mean_squared_error(y_test, y_pred_scaled))
mape_lstm_simple = mean_absolute_percentage_error(y_test, y_pred_scaled)
r2_lstm_simple = r2_score(y_test, y_pred_scaled)
print(f"R^2 is {r2_lstm_simple} \n RMSE is {rmse_lstm_simple} \n MAPE is {mape_lstm_simple}")

In [80]:
y_test

array([[84447]])

In [89]:
y_pred_scaled

array([[88273.33]], dtype=float32)

In [90]:
b = np.array([y_test.item()])

In [91]:
b

array([84447])